In [1]:
import pandas as pd
import numpy as np

In [2]:
def calculate_proximity(df, step=0.1):
    '''
    This function takes a Pandas DataFrame and calculates a weighted proximity around binary data points in both directions with a chosen step,
    e.g. if a data point represents if a certain day is a holiday (1 = holiday, 0 = not a holiday) and step is set to be 0.1,
    then 9 days before and 9 days after the holiday will recieve a value in range [0.1, 0.9] depending on how close they're to the specified day
    
    args:
        df (pd.Series or pd.DataFrame): Pandas DataFrame or Pandas Series with columns where this function should be applied, all columns should have binary values (0 or 1)
        step (float): a value to use when decrementing proximity value
    '''
    # This line transposes a row into a column if a single pd.Series is passed into the function instead of pd.DataFrame with multiple columns
    values = df.values[..., np.newaxis].astype(np.float32) if df.values.ndim == 1 else df.values.astype(np.float32)

    forward_pass = values.copy()
    backward_pass = values[::-1].copy()

    for i in range(1, len(values)):
        mask_forward = (forward_pass[i - 1, :] > 0) & (forward_pass[i, :] != 1)
        forward_pass[i, mask_forward] = (forward_pass[i - 1, mask_forward] - step).round(1)
        mask_backward = (backward_pass[i - 1, :] > 0) & (backward_pass[i, :] != 1)
        backward_pass[i, mask_backward] = (backward_pass[i - 1, mask_backward] - step).round(1)

    new_df = pd.DataFrame(np.maximum(forward_pass, backward_pass[::-1]))
    return new_df

In [3]:
df = pd.DataFrame([[0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
                   [1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]]).T
calculate_proximity(df)

,0,1
0,0.7,1.0
1,0.8,0.9
2,0.9,0.8
3,1.0,0.7
4,0.9,0.6
5,0.9,0.5
6,1.0,0.5
7,0.9,0.6
8,0.8,0.7
9,0.7,0.8
